In [0]:
from ipywidgets import Image
from IPython import display
gif = open('loading.gif', 'rb')
image = gif.read()
widget = Image(value=image, format='gif', width=768, height=576)
display.display(widget)

In [1]:
import pandas as pd
import re
from arcgis.learn import prepare_data
import geopy
from geopy.geocoders import Nominatim, Photon
from geopy.extra.rate_limiter import RateLimiter
import json_lines
from facebook_scraper import get_posts
from json import loads
from requests import post as po
from neurotpr import geoparse
import ast
from functools import partial
from ipyleaflet import LayerGroup, Marker, Map, basemaps, WidgetControl
from ipywidgets import Button
json_path = 'bkk.jsonl'
from silence_tensorflow import silence_tensorflow
silence_tensorflow()
# !pip install 'h5py<3.0.0'

Using TensorFlow backend.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [3]:
txt = open(json_path, "w")
i = 0
for post in get_posts('bkkinfobudapest', pages=100):
    if len(str(post['text'].find('Baleset'))) == 1:
        i += 1
        txt.write(
        ''.join(('{"id": ', str(i), ', "text": "', post['text'][2:post['text'].rfind('\nBKK.HU\n')].replace('.\n', ''),
               '", "meta": {}, "annotation_approver": null, "comments": [], "labels": []}', '\n')))
txt.close()

In [2]:
def english(text):
    HEADERS = {
    "Host": "www.bing.com",
    "User-Agent": "Mozilla/5.0 (Windows NT 6.1; WOW64; rv:52.0) Gecko/20100101 Firefox/52.0",
    "Accept": "*/*",
    "Accept-Language": "en-US,en;q=0.5",
    "Accept-Encoding": "gzip, deflate",
    "Referer": "https://www.bing.com/",
    "Content-Type": "application/x-www-form-urlencoded",
    "Connection": "keep-alive"
    }
    PARAMS = {'IG' : '839D27F8277F4AA3B0EDB83C255D0D70', 'IID' : 'translator.5033.3'}
    text = text.replace(' M ', ' ')
    text = re.sub(r'(?<!^)(?<!\. )[A-Z\u00C0-\u00DC][a-z\u00C0-\u017F]+', lambda x: '+'+''.join(
        [c.upper() if index % 2 == 0 else c.lower() for index, c in enumerate(x.group(0))]), text)
    request = po("https://www.bing.com/ttranslatev3", headers=HEADERS, params=PARAMS, data={'text': str(text), 'fromLang': 'hu', 'to': 'en'})
    if request.status_code < 400:
        return(loads(request.text)[0]["translations"][0]["text"])
    return(text)

In [3]:
with open("bkk.jsonl", "rb") as file:
    for row in json_lines.reader(file):
        pass
data = prepare_data(path=json_path, batch_size=int(row['id'])-1, val_split_pct=0.01,
                    class_mapping={'address_tag':'Address'}, dataset_type='ner_json')
news = list([english(o[0]) for o in data.data.train_ds.data])
topos = []
geoparse.load_model("../PretrainedModel/")
for text in news:
    topos.append(ast.literal_eval(geoparse.topo_recog(re.sub(r'(\+\w+)', lambda x: ''.join(
        [c.lower() if index != 1 and c.isupper() else c for index, c in enumerate(
        x.group(0))]), text).replace('+', '').replace('wharf', 'Square').replace('sq','Sq').replace('str','Str').replace('bou','Bou').replace('roa','Roa'))))

/srv/conda/envs/notebook/lib/python3.7/site-packages/arcgis/learn/models/_arcgis_model.py:336: UserWarning: Cuda is not available
  warnings.warn("Cuda is not available")
                1. If your data has an address field, pass your address field name as address tag in class mapping 
                e.g. - data=prepare_data(dataset_type=ds_type,path=training_data_folder,
			                    class_mapping={address_tag:address_field_name})
                2. Else no action is required, if your data does not have any address information.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Loaded pretrained NeuroTPR model



In [9]:
df = pd.DataFrame({'location': []})
for row in topos:
    df = df.append({'location': row}, ignore_index='True')
df['extracted'] = df.location.apply(lambda x: [y['location_name'] for y in x if len(y['location_name']) > 5][0])

In [8]:
# news

['Due to an accident, the road leading to +FeRiHeGyI +RePüLőTéRrE was closed at the +RePüLőTéR, D port. Bus 200E is diverted and does not affect +FeRiHeGy train station, +RePüLőTéR, D porta and +RePüLőTéRi +ReNdŐr +IgTheGaTóG stop',
 'There was an accident on the +KöNyVeS +KáLmÁn boulevard towards the +RáKóCzI bridge at +GyÁlI road. Instead of tram 1, you can take a replacement bus between +KöZvÁgÓhÍd H and +NéPlIgEt',
 'There was an accident on the +KöNyVeS +KáLmÁn boulevard towards the +ÁrPáD bridge in front of the +GyÁlI road overpass. Instead of tram 1, you can take a replacement bus between +KöZvÁgÓhÍd and +NéPlIgEt',
 'There was an accident on +VáMhÁz boulevard towards +KáLvIn square at +LóNyAy street. Trams 47 and 49 do not run between +DeÁk +FeReNc square and +SzEnT +GeLlÉrT tér - +ArtificialEgYeTeM. We recommend using the 72M trolleybus, bus 9 and metro line M4. Buses 15 and 115 are diverted and do not affect +LóNyAy Street, +CzUcZoR Street and +TöRöK +PáL Utca',
 'There was a

In [6]:
def geocode_locations(processed_df, city, region, address_col):
    # creating address with city and region
    add_miner = processed_df[address_col].apply(lambda x: x+f', {city} '+f', {region}')
    locators = [Nominatim(user_agent="Twitterbot/1.0"), Photon(user_agent = "Twitterbot/1.0")]
    geocode1 = RateLimiter(locators[0].geocode, min_delay_seconds=1)
    geocode2 = RateLimiter(locators[1].geocode, min_delay_seconds=1)
    processed_df['location'] = processed_df[address_col].apply(lambda x: x+f', {city} '+f', {region}').apply(
        partial(geocode2, limit=1, location_bias=(47.5, 19.1)))
   
    def recode(row):
        if (not row.location) or (str(tuple(row.location)[0])[:2]!='47'):
            row['location'] = geocode1(str(row['extracted']) + f', {city} ' + f', {region}')
        return(row)
    
    processed_df = processed_df.apply(recode, axis=1)
    processed_df['point'] = processed_df['location'].apply(lambda loc: tuple(loc.point) if loc else None)
        
    return processed_df

In [7]:
def prepare_sdf(processed_df):
    processed_df = processed_df.query('point == point')
    sdf = processed_df.reset_index(drop=True)
    sdf['geo_x_y'] = sdf['point'].apply(lambda x: (str(x[1]) if x is not None else '') + ',' + str(x[0]) if x is not None else '')
    sdf = pd.DataFrame.spatial.from_df(sdf, address_column='geo_x_y') # adding geometry to the dataframe
    sdf.drop(['location','geo_x_y','point'], axis=1, inplace=True)
    return sdf

In [4]:
def refresh_marks(button):
    txt = open(json_path, "w")
    i = 0
    for post in get_posts('bkkinfobudapest', pages=100):
        if len(str(post['text'].find('Baleset'))) == 1:
            i += 1
            txt.write(
            ''.join(('{"id": ', str(i), ', "text": "', post['text'][2:post['text'].rfind('\nBKK.HU\n')].replace('.\n', ''),
                   '", "meta": {}, "annotation_approver": null, "comments": [], "labels": []}', '\n')))
    txt.close()
    with open(json_path, "rb") as file:
        for row in json_lines.reader(file):
            pass
    data = prepare_data(path=json_path, batch_size=int(row['id'])-1, val_split_pct=0.01,
                    class_mapping={'address_tag':'Address'}, dataset_type='ner_json')
    news = list([english(o[0]) for o in data.data.train_ds.data])
    topos = []
    for text in news:
        topos.append(ast.literal_eval(geoparse.topo_recog(re.sub(r'(\+\w+)', lambda x: ''.join([c.lower() if index != 1 and c.isupper() else c for index, c in enumerate(
            x.group(0))]), text).replace('+', '').replace('wharf', 'Square').replace('sq','Sq').replace('str','Str').replace('bou','Bou').replace('roa','Roa'))))
    df = pd.DataFrame({'location': []})
    for row in topos:
        df = df.append({'location': row}, ignore_index='True')
    df['extracted'] = df.location.apply(lambda x: [y['location_name'] for y in x if len(y['location_name']) > 5][0])
    processed_df = geocode_locations(df.tail(100), 'Budapest', 'Hungary', 'extracted')
    [marks.remove_layer(m) for m in marks.layers]
    processed_df = processed_df.drop_duplicates(subset=['extracted'], keep='first').reset_index(drop=True)
    for idx in processed_df.index:
        row = processed_df.iloc[idx]
        if row.point is not None:
            mark = Marker(location=(row.point[0], row.point[1]), title=str(row["extracted"]), radius=5, color="red")
            marks.add_layer(mark)

In [10]:
processed_df = geocode_locations(df.tail(100), city='Budapest', region='Hungary', address_col='extracted')
m = Map(center=(47.51, 19.04), zoom=11.5, basemap=basemaps.CartoDB.Positron)
marks = LayerGroup()
for idx in processed_df.index:
    row = processed_df.iloc[idx]
    if row.point is not None:
        mark = Marker(
        location=(row.point[0], row.point[1]),
        title=str(row["extracted"]),
        radius=5)
        marks.add_layer(mark)
m.add_layer(marks)
refresh_button = Button(description='Refresh')
m.add_control(WidgetControl(widget=refresh_button, position='bottomright'))
refresh_button.on_click(refresh_marks)
widget.close()
m

Map(center=[47.51, 19.04], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_o…